# Naive Bayes Tutorial Part 1: Predicting survival from titanic crash

## Librarie and Dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


## Exploratory Data Analysis

In [3]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [4]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [6]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [7]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**

In [8]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [9]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [10]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [11]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


## Train and Test Split

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.25)

In [37]:
len(X_train)

668

In [38]:
len(X_test)

223

## Creating the Model

In [39]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [40]:
model.fit(X_train,y_train)

GaussianNB()

In [41]:
model.score(X_test,y_test)

0.7399103139013453

In [42]:
X_test[0:10]

,Pclass,Age,Fare,female
297,1,2.000000,151.5500,1
642,3,2.000000,27.9000,1
182,3,9.000000,31.3875,0
863,3,29.699118,69.5500,1
715,3,19.000000,7.6500,0
577,1,39.000000,55.9000,1
362,3,45.000000,14.4542,1
387,2,36.000000,13.0000,1
402,3,21.000000,9.8250,1
719,3,33.000000,7.7750,0


In [43]:
y_test[0:10]

297    0
642    0
182    0
863    0
715    0
577    1
362    0
387    1
402    0
719    0
Name: Survived, dtype: int64

In [44]:
model.predict(X_test[0:10])

array([1, 1, 0, 1, 0, 1, 1, 1, 1, 0], dtype=int64)

In [45]:
model.predict_proba(X_test[:10])

array([[5.14792633e-06, 9.99994852e-01],
       [2.52455274e-01, 7.47544726e-01],
       [9.52962642e-01, 4.70373576e-02],
       [1.46870144e-01, 8.53129856e-01],
       [9.66426783e-01, 3.35732175e-02],
       [3.46152933e-02, 9.65384707e-01],
       [4.12971332e-01, 5.87028668e-01],
       [2.38361575e-01, 7.61638425e-01],
       [3.82138741e-01, 6.17861259e-01],
       [9.71714982e-01, 2.82850175e-02]])

**Calculate the score using cross validation**

In [46]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.80597015, 0.82089552, 0.79850746, 0.78195489, 0.73684211])